## Predicting All Stars Based of a Full Year of Baseball
Team Members: Chris Spartz, Kevin Gnaster, Zach Josten <br />
Team Number: 31

In [1]:
import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import scipy.stats as st
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
import statsmodels.api as sm
import math

In [16]:
star = pd.read_excel("FinalDatasetMLBallStar 11.25.21.xlsx")
star.drop(columns=['xwOBA'], inplace=True)
star.head()

,Season,YearName,Name,Team,G,PA,HR,R,RBI,SB,...,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid,All Star Starter?
0,2019,2019Mike Trout,Mike Trout,LAA,134,600,45,110,104,11,...,0.438,0.645,0.436,177.0,7.1,65.9,-0.3,8.4,10155,1
1,2019,2019Alex Bregman,Alex Bregman,HOU,156,690,41,122,112,5,...,0.423,0.592,0.418,167.0,-2.1,57.3,4.8,8.4,17678,1
2,2019,2019Christian Yelich,Christian Yelich,MIL,130,580,44,100,97,30,...,0.429,0.671,0.442,174.0,8.5,65.2,-3.9,7.8,11477,1
3,2019,2019Cody Bellinger,Cody Bellinger,LAD,156,660,47,121,115,15,...,0.406,0.629,0.415,161.0,1.4,54.2,3.4,7.7,15998,1
4,2019,2019Marcus Semien,Marcus Semien,OAK,162,747,33,123,92,10,...,0.369,0.522,0.373,138.0,1.7,37.7,14.1,7.6,12533,0


In [19]:
star2 = star.dropna(how='any',axis=0)
star2.mode()

,Season,YearName,Name,Team,G,PA,HR,R,RBI,SB,...,OBP,SLG,wOBA,wRC+,BsR,Off,Def,WAR,playerid,All Star Starter?
0,2018.0,1999Alex Gonzalez,Chris Young,- - -,32.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-100.0,0.0,-0.3,0.1,0.0,639.0,0.0
1,2019.0,1999Bobby Jones,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,1999Brian Hunter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1999Freddy Garcia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1999Kevin Brown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,2000Alex Gonzalez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,2000Bobby Jones,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,2000Brian Hunter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,2000Carlos Hernandez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,2000Kevin Brown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
